In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
#Se carga el archivo de restaurantes ya limpio para poder obtener las ids de google
# y asi crear un data set solo las reviews de los restaurantes
df_restaurantes= pd.read_parquet('Datos limpios/restaurantes_clean.parquet')
gmaps=df_restaurantes['gmap_id'].astype(str).tolist()

In [3]:
#Hago un diccionarion con las rutas de los archivos de las reviews
ruta_review_estados={
    'NY': 'Datasets/Google Maps/reviews-estados/review-New_York',
    'CA': 'Datasets/Google Maps/reviews-estados/review-California',
    'TX': 'Datasets/Google Maps/reviews-estados/review-Texas',
    'FL': 'Datasets/Google Maps/reviews-estados/review-Florida',
    'PA': 'Datasets/Google Maps/reviews-estados/review-Pennsylvania',

}

In [5]:
reviews=[]
#Recorro todos los archivos y solo me quedo con las reviews de los restaurantes
for ruta in ruta_review_estados.values():
#Por cada ruta se recorren todos los archvivos
    for archivo in os.listdir(ruta):
        ruta_archivo=os.path.join(ruta, archivo)
        with open(ruta+'/'+archivo,'r') as file:
            for linea in file:
                data = json.loads(linea)
                reviews.append(data)
            
       

In [26]:
#Ahora transformamos el diccionario en un data frame
df_reviews=pd.DataFrame(reviews)

In [21]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12946824 entries, 0 to 12946823
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 790.2+ MB


In [27]:
#Eliminamos las columnas que no nos interesan
df_reviews.drop(columns=['pics','resp'],inplace=True)

In [28]:
#Unimos los dos dataframes
df_reviews=df_reviews.merge(df_restaurantes['gmap_id'],on='gmap_id',how='left')

In [29]:
#Reordenamos las columnas 
df_reviews=df_reviews[['gmap_id','name','rating','time','text']]

In [31]:
#Modificamos el formato de la fecha
df_reviews['time']=pd.to_datetime(df_reviews['time'], unit='ms')

In [35]:
#Nos quedamos con las reviews de los años 2016 a 2021
df_reviews=df_reviews[(df_reviews['time'].dt.year>=2016) & (df_reviews['time'].dt.year<=2021)]

In [37]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12723874 entries, 0 to 12946823
Data columns (total 5 columns):
 #   Column   Dtype         
---  ------   -----         
 0   gmap_id  object        
 1   name     object        
 2   rating   int64         
 3   time     datetime64[ns]
 4   text     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 582.5+ MB


In [38]:
df_reviews.head()

,gmap_id,name,rating,time,text
0,0x89c25fc9494dce47:0x6d63c807b59a55,Alvin Martinez,5,2020-10-23 23:13:15.361,I'm late to posting this but this store especi...
1,0x89c25fc9494dce47:0x6d63c807b59a55,Johnnie Jackson,1,2021-05-04 19:37:17.403,Very dissatisfied I did not get my phone the p...
2,0x89c25fc9494dce47:0x6d63c807b59a55,Manie Blazer,5,2020-08-14 19:01:02.039,Excellent very well done with professional car...
3,0x89c25fc9494dce47:0x6d63c807b59a55,Fashion Fiinds,5,2018-12-02 18:04:22.044,Basing my review strictly on the service I rec...
4,0x89c25fc9494dce47:0x6d63c807b59a55,Andres Rieloff,1,2020-08-13 00:38:17.718,Bad! Disorganized. I'm being totally honest. I...


In [41]:
df_reviews.to_parquet('Datos limpios/reviews_restaurantes.parquet')